In [41]:
import vrep 
import sys
import time 
import numpy as np
from tank import *
from skfuzzy import control as ctrl

## Połączenie z V-REP

In [42]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)

#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)


Connected to remote API server


In [43]:
def read_proximity_sensor(sensor_name: str):
    sensors = {
        "EN": 0,
        "ES": 1,
        "NE": 2,
        "NW": 3,
        "SE": 4,
        "SW": 5,
        "WN": 6,
        "WS": 7
    }
    _,detectionState,detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,proximity_sensors_handles[sensors[sensor_name]],vrep.simx_opmode_buffer)
    return detectionState, detectedPoint

## Sprawdzanie wartości czujnikow

In [44]:
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        print(f'{sensor_name}: {detectionState}, {np.linalg.norm(detectedPoint):.2f}')

EN: False, 0.00
ES: False, 0.00
NE: False, 0.00
NW: False, 0.00
SE: False, 0.00
SW: False, 0.00
WN: False, 0.00
WS: False, 0.00


## Dojechanie do optymalnego miejsca do zaparkowania

In [45]:
x_speeds = np.arange(-3, 8, 1)
x_distances = np.arange(0, 5, 1)

EN_dist = ctrl.Antecedent(x_distances, 'EN_dist')
ES_dist = ctrl.Antecedent(x_distances, 'ES_dist')
SE_dist = ctrl.Antecedent(x_distances, 'SE_dist')

target_rotation_speed = ctrl.Consequent(x_speeds, 'target_velocity')

EN_dist.automf(3)
ES_dist.automf(3)
SE_dist.automf(5)
target_rotation_speed.automf(5)

rules = (
    ctrl.Rule(EN_dist['poor'] & ES_dist['poor'] & SE_dist['decent'], target_rotation_speed['mediocre']),
    ctrl.Rule((EN_dist['average'] | EN_dist['good']) & (ES_dist['average'] | ES_dist['good']) &
              (SE_dist['poor'] | SE_dist['mediocre'] | SE_dist['average'] | SE_dist['good']), target_rotation_speed['good']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

# EN_dist.view(sim=driver)
# SE_dist.view(sim=driver)
# target_rotation_speed.view(sim=driver)

while True:
    detectionState,EN_detectedPoint = read_proximity_sensor('EN')
    if detectionState:
        EN_curr_dist = np.linalg.norm(EN_detectedPoint)
    else:
        EN_curr_dist = 100


    detectionState,ES_detectedPoint = read_proximity_sensor('ES')
    if detectionState:
        ES_curr_dist = np.linalg.norm(ES_detectedPoint)
    else:
        ES_curr_dist = 100


    detectionState,SE_detectedPoint = read_proximity_sensor('SE')
    if detectionState:
        SE_curr_dist = np.linalg.norm(SE_detectedPoint)
    else:
        SE_curr_dist = 100

    driver.input['EN_dist'] = EN_curr_dist
    driver.input['ES_dist'] = ES_curr_dist
    driver.input['SE_dist'] = SE_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values EN:{EN_curr_dist}, ES:{ES_curr_dist}, SE:{SE_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rEN_dist: {EN_curr_dist:.2f}, ES_dist: {ES_curr_dist:.2f}, SE_dist: {SE_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        target = 0
        tank.forward(target)
        break

    tank.forward(target)


EN_dist: 0.81, ES_dist: 0.87, SE_dist: 2.92, target_velocity: 0.11909610

## Obrót celujący do miejsca do zaparkowania

In [46]:
x_speeds = np.arange(-2, 8, 1)
x_distances = np.arange(0, 5, 1)

ES_dist = ctrl.Antecedent(x_distances, 'ES_dist')
SE_dist = ctrl.Antecedent(x_distances, 'SE_dist')
SW_dist = ctrl.Antecedent(x_distances, 'SW_dist')

target_rotation_speed = ctrl.Consequent(x_speeds, 'target_velocity')

ES_dist.automf(3)
SE_dist.automf(3)
SW_dist.automf(5)
target_rotation_speed.automf(5)

rules = (
    ctrl.Rule(ES_dist['poor'] & SE_dist['poor'] & SW_dist['decent'], target_rotation_speed['mediocre']),
    ctrl.Rule((SE_dist['average'] | SE_dist['good']) & (ES_dist['average'] | ES_dist['good']) &
              (SW_dist['poor'] | SW_dist['mediocre'] | SW_dist['average'] | SW_dist['good']), target_rotation_speed['good']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    _,detectionState,SE_detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,proximity_sensors_handles[0],vrep.simx_opmode_buffer)
    if detectionState:
        SE_curr_dist = np.linalg.norm(SE_detectedPoint)
    else:
        SE_curr_dist = 100


    _,detectionState,ES_detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,proximity_sensors_handles[1],vrep.simx_opmode_buffer)
    if detectionState:
        ES_curr_dist = np.linalg.norm(ES_detectedPoint)
    else:
        ES_curr_dist = 100


    _,detectionState,SW_detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,proximity_sensors_handles[5],vrep.simx_opmode_buffer)
    if detectionState:
        SW_curr_dist = np.linalg.norm(SW_detectedPoint)
    else:
        SW_curr_dist = 100

    driver.input['ES_dist'] = ES_curr_dist
    driver.input['SE_dist'] = SE_curr_dist
    driver.input['SW_dist'] = SW_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values SE:{SE_curr_dist}, ES:{ES_curr_dist}, SW:{SW_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rSE_dist: {SE_curr_dist:.2f}, ES_dist: {ES_curr_dist:.2f}, SW_dist: {SW_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        target = 0
        tank.turn_left(target)
        break

    tank.turn_left(target)

SE_dist: 1.00, ES_dist: 0.88, SW_dist: 3.01, target_velocity: 0.37

## Wycofanie do okolic miejsca parko\wania

In [47]:
x_speeds = np.arange(-2, 5, 1)
x_distances = np.arange(0.5, 3, 1)

EN_dist = ctrl.Antecedent(x_distances, 'EN_dist')
ES_dist = ctrl.Antecedent(x_distances, 'ES_dist')
SW_dist = ctrl.Antecedent(x_distances, 'SW_dist')

target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

EN_dist.automf(3)
ES_dist.automf(3)
SW_dist.automf(5)
target_speed.automf(5)

rules = (
    ctrl.Rule(EN_dist['poor'] & ES_dist['poor'] & SW_dist['average'], target_speed['poor']),
    ctrl.Rule((EN_dist['good'] | EN_dist['average']) & (ES_dist['good'] | ES_dist['average']) & (SW_dist['good'] | SW_dist['poor']), target_speed['decent']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:

    detectionState, EN_detectedPoint = read_proximity_sensor('EN')
    if detectionState:
        EN_curr_dist = np.linalg.norm(EN_detectedPoint)
    else:
        EN_curr_dist = 100

    detectionState, ES_detectedPoint = read_proximity_sensor('ES')
    if detectionState:
        ES_curr_dist = np.linalg.norm(ES_detectedPoint)
    else:
        ES_curr_dist = 100

    detectionState, SW_detectedPoint = read_proximity_sensor('SW')
    if detectionState:
        SW_curr_dist = np.linalg.norm(SW_detectedPoint)
    else:
        SW_curr_dist = 100

    driver.input['EN_dist'] = EN_curr_dist
    driver.input['ES_dist'] = ES_curr_dist
    driver.input['SW_dist'] = SW_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values SE:{SE_curr_dist}, SW:{SW_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rSE_dist: {SE_curr_dist:.2f}, SW_dist: {SW_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        tank.stop()
        break

    tank.turn_left(2.5)
    tank.backward(target)

SE_dist: 1.00, SW_dist: 2.01, target_velocity: 0.48

In [48]:
x_speeds = np.arange(-2, 5, 1)
x_distances = np.arange(0.65, 3.65, 1)

SW_dist = ctrl.Antecedent(x_distances, 'SW_dist')
SE_dist = ctrl.Antecedent(x_distances, 'SE_dist')

target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

SW_dist.automf(3)
SE_dist.automf(3)
target_speed.automf(5)

rules = (
    ctrl.Rule(SE_dist['poor'] | SW_dist['poor'], target_speed['poor']),
    ctrl.Rule((SE_dist['good'] | SE_dist['average']) & (SW_dist['good'] | SW_dist['average']), target_speed['decent']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    detectionState, SE_detectedPoint = read_proximity_sensor('SE')
    if detectionState:
        SE_curr_dist = np.linalg.norm(SE_detectedPoint)
    else:
        SE_curr_dist = 100

    detectionState, SW_detectedPoint = read_proximity_sensor('SW')
    if detectionState:
        SW_curr_dist = np.linalg.norm(SW_detectedPoint)
    else:
        SW_curr_dist = 100

    driver.input['SE_dist'] = SE_curr_dist
    driver.input['SW_dist'] = SW_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values SE:{SE_curr_dist}, SW:{SW_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rSE_dist: {SE_curr_dist:.2f}, SW_dist: {SW_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        tank.stop()
        break

    tank.turn_right(0.75 * target)
    tank.backward(target)

SE_dist: 100.00, SW_dist: 0.93, target_velocity: 0.49

In [49]:
x_speeds = np.arange(-2, 5, 1)
x_distances = np.arange(0.65, 3.65, 1)

SW_dist = ctrl.Antecedent(x_distances, 'SW_dist')
SE_dist = ctrl.Antecedent(x_distances, 'SE_dist')
NE_dist = ctrl.Antecedent(x_distances, 'NE_dist')
NW_dist = ctrl.Antecedent(x_distances, 'NW_dist')

target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

SW_dist.automf(3)
SE_dist.automf(3)
NE_dist.automf(3)
NW_dist.automf(3)
target_speed.automf(5)

rules = (
    ctrl.Rule((SE_dist['average'] | SW_dist['average']) & (NE_dist['average'] | NW_dist['average']), target_speed['poor']),
    ctrl.Rule((SE_dist['good'] | SE_dist['poor']) &
              (SW_dist['good'] | SW_dist['poor']) &
              (NE_dist['good'] | NE_dist['poor']) &
              (NW_dist['good'] | NW_dist['poor']), target_speed['decent']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    detectionState, SE_detectedPoint = read_proximity_sensor('SE')
    if detectionState:
        SE_curr_dist = np.linalg.norm(SE_detectedPoint)
    else:
        SE_curr_dist = 100

    detectionState, SW_detectedPoint = read_proximity_sensor('SW')
    if detectionState:
        SW_curr_dist = np.linalg.norm(SW_detectedPoint)
    else:
        SW_curr_dist = 100

    detectionState, NE_detectedPoint = read_proximity_sensor('NE')
    if detectionState:
        NE_curr_dist = np.linalg.norm(NE_detectedPoint)
    else:
        NE_curr_dist = 100

    detectionState, NW_detectedPoint = read_proximity_sensor('NW')
    if detectionState:
        NW_curr_dist = np.linalg.norm(NW_detectedPoint)
    else:
        NW_curr_dist = 100

    driver.input['SE_dist'] = SE_curr_dist
    driver.input['SW_dist'] = SW_curr_dist
    driver.input['NE_dist'] = NE_curr_dist
    driver.input['NW_dist'] = NW_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values SE:{SE_curr_dist}, SW:{SW_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rSE_dist: {SE_curr_dist:.2f}, SW_dist: {SW_curr_dist:.2f}, NE_dist: {NE_curr_dist:.2f}, NW_dist: {NW_curr_dist:.2f} target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        tank.stop()
        break

    tank.turn_right(3*target)
    tank.forward(target)

SE_dist: 1.44, SW_dist: 1.68, NE_dist: 0.76, NW_dist: 100.00 target_velocity: -0.095